In [ ]:
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np
import random
import re
import os
import faiss
from sentence_transformers import SentenceTransformer
import sys
sys.path.append("../../../")
sys.path.append("../")

from utils.helper_functions import clean_title
from BM25.BM25_retriever import BM25Retriever
#from helper_functions import remove_punctuation, lemmatization#, remove_stopwords
from retriever import SAVIARetriever

%load_ext autoreload
%autoreload 2

In [ ]:
client = MongoClient("mongodb://mongo_db:27017")
db = client["SAVIA"]
coll_leggi_regionali = db["leggiRegionali"]
print(sorted(db.list_collection_names()))

### Load dataset

In [ ]:
dataset_folder = "../../../../datasets/instruction_datasets/instruction_dataset_v2"

with open(os.path.join(dataset_folder, "IB_dataset_train.json"), "r") as f:
    IB_dataset_train = json.load(f)

with open(os.path.join(dataset_folder, "IB_dataset_test.json"), "r") as f:
    IB_dataset_test = json.load(f)

### Load NER model

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("DeepMount00/GLiNER_ITA_LARGE", device="cuda:0")
labels = ["anno"]

### NER

In [ ]:
ind = random.randint(0, len(IB_dataset_train) - 1)
#ind = 64
print(ind)
question = IB_dataset_train[ind]['question']
context = IB_dataset_train[ind]['context']
gt = context[0]['legge'].upper().replace(" , ", ", ")

print(question)
print(gt)

entities = model.predict_entities(question, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [ ]:
# ind wrong [1981 1356 2132 ]   test dataset

#[ 1577  ]         train dataset

In [ ]:
retriever = SAVIARetriever()

In [ ]:
#retriever._id_leggi

In [ ]:


retrieved_item = retriever.create_prompt(question)
retrieved_item
#print(retrieved_item['legge'])

#if retrieved_item is not None and retrieved_item.upper() != gt:
#    print("wrong", retrieved_item, "  -----   ", gt)
#else:
#    print("ok")

In [ ]:
coll_atti_giunta_regionale = db["attiGiuntaRegionale"]

In [ ]:
res = coll_atti_giunta_regionale.find_one({'_id': ObjectId('6633a37d584dc13ec6ca4024')})
res

In [ ]:
res['testo']['testo_atto']

In [ ]:
#res['testo']

In [ ]:
res['_id_atti']

In [ ]:
res = coll_leggi_regionali.find_one({'_id': ObjectId('663f1fa1e9c699e78e2b8a8b')})

In [ ]:
chunk = ' atti attuativi e delle'

question_embeddings = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
vector_store_folder = "../../../../SAVIA_vector_stores"
index = faiss.read_index(os.path.join(vector_store_folder , "leggiRegionaliAllegati" + ".faiss"))

with open(os.path.join("./", "allegati_diciture.json"), "r", encoding="utf-8") as f:
    allegati_diciture = json.load(f)

chunks_list = []

for k, v in allegati_diciture.items():
#    print(k, v)
    for dicitura in v[0:]:
        chunks_list.append(dicitura)

In [ ]:
print("num chunks:", index.ntotal)

In [ ]:
len(chunks_list)

In [ ]:
#index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
#embedding = embedding_model.encode([chunk.lower()])
#faiss.normalize_L2(embedding)
#index.add_with_ids(embedding, np.array([0]))

### Load FAISS indices

In [ ]:
vector_store_folder = "../../../../SAVIA_vector_stores"
index_name = "leggiRegionaliKeywords"
coll_leggi_regionali_keywords = db[index_name]

In [ ]:
leggi_regionali_keywords_index = faiss.read_index(os.path.join(vector_store_folder, "leggiRegionaliKeywords.faiss"))

### Merging two indices

In [ ]:
# Assuming db1 and db2 are your two FAISS vectorstores
#index1 = db1.faiss_index
#index2 = db2.faiss_index

# Create a new FAISS index with the same dimension as the original indexes
#new_index = FAISS.create_faiss_index(index1.d)

# Add vectors from both indexes to the new index
#new_index.add(np.vstack((index1.reconstruct(i) for i in range(index1.ntotal))))
#new_index.add(np.vstack((index2.reconstruct(i) for i in range(index2.ntotal))))

# Create a new FAISS vectorstore with the merged index
#new_db = FAISS(new_index, db1.embeddings)

### Retrieve content with BM25

In [ ]:
list_all_laws = []
corpus = []

for ind, doc in enumerate(coll_leggi_regionali.find()[0:]):
    titolo_orig = doc['titolo']
#    titolo = titolo_orig
    titolo = clean_title(titolo_orig).lower()
    out_dict = {"titolo": titolo, "legge": doc["legge"], 
                          "testo": doc['testo'].replace("\n", " ").lower(),#, "numero": doc["numero"],
                        "legge": doc['legge']}#, "numeroAnno": doc["numeroAnno"]}

    if 'articoli' in doc.keys():
        articoli = list(doc['articoli'].values())
        articoli_str = " ".join([x for x in articoli[0:2]]).lower()
        
        out_dict["articoli"] = articoli_str

    list_all_laws.append(out_dict)

    corpus.append(" ".join(list(out_dict.values())))

In [ ]:
bm25_retriever = BM25Retriever(corpus, top_n = 30, threshold = None)

In [ ]:
ind = random.randint(0, len(IB_dataset_train) - 1)
#ind = 825
print(ind)
question = IB_dataset_train[ind]['question']
context = IB_dataset_train[ind]['context']
gt = context[0]['legge'].upper().replace(" , ", ", ")

print(question)

In [ ]:
top_n_inds, top_n_sim = bm25_retriever.retrieve(question)

for ind_sim, ind in enumerate(top_n_inds):
    pred = list_all_laws[ind]['legge'].upper().replace(" , ", ", ")
#    print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 

#    print("PRED: {}".format(pred), "  ----   GT: {}".format(gt))
#    print("titolo:", list_all_laws[ind]['titolo'].upper())

    if pred == gt:
        print("True document")
        print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 

        print("PRED: {}".format(pred), "  ----   GT: {}".format(gt))
        print("titolo:", list_all_laws[ind]['titolo'].upper())
#    print("--------------")

In [ ]:
#top_m = 5
#question_embedding = embedding_model.encode([question.lower()])
#faiss.normalize_L2(question_embedding)

#similarity, item_ids = leggi_regionali_keywords_index.search(question_embedding, top_m)
#similarity = similarity[0]
#item_ids = item_ids[0]

#for ind_item, item_id in enumerate(item_ids[0:]):
#    print(item_id)
#    print("PRED ANSWER:", list_texts[item_id], " - sim:", round(similarity[ind_item], 3))

#    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
#    _id_legge = res_chunk['_id_legge']
#    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
#    legge = res_legge['legge']
#    print("legge:", legge, "- sim:", round(similarity[ind_item], 3))

In [ ]:
retrieved_item

In [ ]:
chunk = 'i atti della legge regionale n'

question_embeddings = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(question_embeddings)

In [ ]:
similarity, item_ids = retriever.leggi_regionali_allegati_index.search(question_embeddings, 1)
#similarity = similarity[0]
#item_ids = item_ids[0]
print(similarity, item_ids)

In [ ]:
question_embeddings.shape

In [ ]:
index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
embedding = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(embedding)
index.add_with_ids(embedding, np.array([0]))

In [ ]:
question_embeddings = embedding_model.encode(["provvedimenti attuativi".lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
question

In [ ]:
good = 0
bad = 0

for ind in range(0, len(IB_dataset_train)):
#for ind in range(0, 1000):
    question = IB_dataset_train[ind]['question']
    context = IB_dataset_train[ind]['context']
    gt = context[0]['legge'].upper().replace(" , ", ", ")

    retrieved_item = retriever.retrieve(question)
    #print(retrieved_item)
    
    if retrieved_item.upper() != gt:
#        print("wrong", retrieved_item, "  -----   ", gt)
        bad += 1
    else:
#        print("ok")
        good += 1

print("accuracy:", good/(good+bad))

In [ ]:
#results

In [ ]:
retrieved_item = retriever.retrieve(question)
retrieved_item

In [ ]:
chunk = "RETE ESCURSIONISTICA DELL'EMILIA-ROMAGNA E VALORIZZAZIONE DELLE ATTIVITÀ ESCURSIONISTICHE"

In [ ]:
index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
embedding = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(embedding)
index.add_with_ids(embedding, np.array([0]))

In [ ]:
question_embeddings = embedding_model.encode([question.lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
res = coll_leggi_regionali.find_one({"_id": retriever._id_leggi[579]})
res['legge']

In [ ]:
list(res['testo_orig']['articoli'].values())[0:2]

In [ ]:
retriever._id_leggi[579]

In [ ]:
res = coll_leggi_regionali.find_one({"_id": retriever._id_leggi[579]})
res['legge']

In [ ]:
top_m = 1
question_embedding = embedding_model.encode(texts)
faiss.normalize_L2(question_embedding)

similarity, item_ids = leggi_regionali_keywords_index.search(question_embedding, top_m)
similarity = similarity[:, 0]
item_ids = item_ids[:, 0]

max_similarity_id = np.argmax(similarity)
best_similarity = similarity[max_similarity_id]
best_item_id = item_ids[max_similarity_id]


    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
    _id_legge = res_chunk['_id_legge']
    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
    legge = res_legge['legge']
    print("legge:", legge, "- sim:", round(best_similarity[ind_item], 3))

for ind_item, item_id in enumerate(best_item_id[0:]):
#    print(item_id)
#    print("PRED ANSWER:", list_texts[item_id], " - sim:", round(similarity[ind_item], 3))

    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
    _id_legge = res_chunk['_id_legge']
    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
    legge = res_legge['legge']
    print("legge:", legge, "- sim:", round(best_similarity[ind_item], 3))

In [ ]:
best_item_id

In [ ]:
item_ids

In [ ]:
np.argmax(similarity)

### Test search over a subset

In [ ]:
import numpy as np
import faiss

# Set random seed for reproducibility
np.random.seed(0)

# Create a set of 5 small binary vectors
vectors = np.array([[1., 0., 1.],
                    [0., 1., 0.],
                    [1., 1., 0.],
                    [0., 0., 1.],
                    [1., 0., 0.]]).astype(np.float32)

# Initialize an index with the Hamming distance measure
index = faiss.IndexFlatL2(vectors.shape[1])

# Add vectors to the index
index.add(vectors)

# Perform a similarity search
query_vector = np.array([[1, 1, 0]], dtype=np.float32)
k = 3  # Number of nearest neighbors to retrieve

distances, indices = index.search(query_vector, k)
print(indices)

In [ ]:
res = np.int64(4)

In [ ]:
type(res)

In [ ]:
filter_ids = np.array([0, 1, 2, 3], dtype = np.int64)
id_selector = faiss.IDSelectorArray(4, filter_ids)

#filtered_distances, filtered_indices = index.search(query_vector, k, params=faiss.SearchParametersIVF(sel=id_selector))
#print(filtered_indices)

In [ ]:
help(faiss.IDSelectorArray)

In [ ]:
type(vectors[0][0])

In [ ]:
coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})

In [ ]:
for doc in coll_leggi_regionali_keywords.find()[240998:240999]:
    print(doc)

In [ ]:
top_n_inds, top_n_sim = retriever.retrieve(question)

for ind_sim, ind in enumerate(top_n_inds):
    print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 
    print("titolo:", list_all_laws[ind]['titolo'].upper())
    print("legge: {}".format(list_all_laws[ind]['legge'].upper()))
          
    print("--------------")